This notebook has been used to create from zero a model based on ResNet architecture. This model is NOT part of the submission on Codalab.
The peculiarities are:
- Based on ResNet architecture
- We applied resizing to 224x224
- We appliead rescaling to [0, 2]
- We didn't apply augmentation techniques during the training phase

In [ ]:
seed = 75

import shutil
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from tensorflow.keras.models import Sequential

from functools import partial

tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [ ]:
dataset = np.load('clean_collection.npz', allow_pickle=True)
data = dataset['data']
labels = dataset['labels']

# Cast string labels into integer values
casted_labels = []
for label in labels:
  if label == "unhealthy":
    casted_labels.append(1)
  elif label == "healthy":
    casted_labels.append(0)
  else:
    raise Exception("Invalid label")

# Expand the labels dimension moving from (x,) to (x, 1), with x cardinality
casted_labels = np.expand_dims(casted_labels, axis=-1)

print("Training-Validation-Test Data Shape:", data.shape)
print("Training-Validation-Test Casted Labels Shape:", casted_labels.shape)

# Inspect the target
print('Counting occurrences of target classes:')
unique, counts = np.unique(casted_labels, return_counts=True)
print(dict(zip(unique, counts)))

# We split the dataset in training_and_validation set and test set
x_train_val, x_test, y_train_val, y_test = train_test_split(data, casted_labels, random_state=seed, test_size=500, stratify = casted_labels)

# Print the shapes of the resulting datasets
print("Training-Validation Data Shape:", x_train_val.shape)
print("Training-Validation Label Shape:", y_train_val.shape)
print("Test Data Shape:", x_test.shape)
print("Test Label Shape:", y_test.shape)

del dataset
del data
del labels
del unique
del counts

In [ ]:
img_augmentation = Sequential(
    [
        tfkl.RandomRotation(factor=0.15),
        tfkl.RandomTranslation(height_factor=0.1, width_factor=0.1),
        tfkl.RandomFlip(),
    ],
    name="img_augmentation",
)

In [ ]:
# Define the default conv. layer
DefaultConv2D = partial(tfkl.Conv2D, kernel_size=3, strides=1, padding="same", kernel_initializer="he_normal",use_bias=False)

class ResidualUnit(tfkl.Layer):
  def __init__(self, filters, strides=1, **kwargs):
    super().__init__()

    # Main model layers
    self.main_layers=[
        DefaultConv2D(filters, strides=strides),
        tfkl.BatchNormalization(),
        tfkl.ReLU(),
        DefaultConv2D(filters),
        tfkl.BatchNormalization()
    ]

    # Only if strides is greater than 1 we add other layers
    self.skip_layers=[]
    if strides > 1:
      self.skip_layers = [
          DefaultConv2D(filters, strides=strides, kernel_size=1),
          tfkl.BatchNormalization()
      ]

  # Returns the layers
  def call(self, inputs):
    Z = inputs
    for layer in self.main_layers:
      Z = layer(Z)
    skip_Z = inputs
    for layer in self.skip_layers:
      skip_Z = layer(skip_Z)
    return tfkl.ReLU()(Z + skip_Z)


In [ ]:
def build_model(denses, units):

  # The input layer
  inputs = tfkl.Input(shape=(96, 96, 3))

  # Perform augmentation (only during training!)
  x = img_augmentation(inputs)

  # Resizing layer used to resize input images
  x = tfkl.Resizing(height = 224, width = 224, crop_to_aspect_ratio = True)(x)

  # Rescaling layer used to rescale input images
  x = tfkl.Rescaling(scale = 2/255, offset = -1)(x)

  # Let's build the network

  x = DefaultConv2D(64, kernel_size = 7, strides = 2)(x)
  x = tfkl.BatchNormalization()(x)
  x = tfkl.ReLU()(x)
  x = tfkl.MaxPool2D(pool_size = 3, strides = 2, padding= "same")(x)

  prev_filters = 64
  for filters in [64] * 2 + [128] * 2 + [256] * 2 + [512] * 2:
    strides = 1 if filters == prev_filters else 2
    ru = ResidualUnit(filters, strides=strides)
    x = ru(x)
    prev_filters = filters

  # Last part
  x = tfkl.GlobalAveragePooling2D()(x)
  if(denses != 0):

    # For each dense layer we specify the number of neurons, the kernel_initializer function, a batch normalization layer and the ReLU activation function
    for ind in range(1, denses + 1):
      x = tfkl.Dense(units=int(units/ind), kernel_initializer=tfk.initializers.HeUniform(seed=seed))(x)
      x = tfkl.BatchNormalization()(x)
      x = tfkl.ReLU()(x)

  # The output layer
  outputs = tfkl.Dense(1, activation="sigmoid", name="pred")(x)

  model = tf.keras.Model(inputs, outputs, name="model")

  model.compile(
    optimizer=tf.keras.optimizers.AdamW(), loss=tfk.losses.BinaryCrossentropy(), metrics=["accuracy"]
  )

  return model

In [ ]:
# Standard parameters

# Define the number of folds for cross-validation
num_folds = 5

# Initialize lists to store training histories, scores, and best epochs
scores = []
best_epochs = []

# Create a KFold cross-validation object
kfold = KFold(n_splits=num_folds, shuffle=True, random_state=seed)

lr_scheduler = tfk.callbacks.ReduceLROnPlateau(
  monitor='val_accuracy',                                                 # Metric to monitor (validation mean squared error in this case)
  patience=5,                                                             # Number of epochs with no improvement after which learning rate will be reduced
  factor=0.9,                                                             # Factor by which the learning rate will be reduced (0.9 in this case)
  mode='min',                                                             # Mode to decide when to reduce learning rate ('min' means reduce when metric stops decreasing)
  min_lr=1e-5                                                             # Minimum learning rate
)

early_stopping = tfk.callbacks.EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    patience=30,
    restore_best_weights=True
)

batch_size = 64
epochs=300

patience = 30

callbacks = [early_stopping, lr_scheduler]

# Loop through each fold
for fold_idx, (train_idx, valid_idx) in enumerate(kfold.split(x_train_val, y_train_val)):

  print("Starting training on fold num: {}".format(fold_idx+1))

  indexTrainingTrue = []
  for i in range(0, len(y_train_val)):
    if i in train_idx:
      indexTrainingTrue.append(True)
    else:
      indexTrainingTrue.append(False)

  # Build a new model for each fold
  model = build_model(1,64)

  # Train the model on the training data for this fold and store the history
  history = model.fit(
    x = x_train_val[indexTrainingTrue],                                       # Select only the images that compose the training set
    y = y_train_val[indexTrainingTrue],                                       # Select only the images that compose the validation set
    validation_data=(x_train_val[np.logical_not(indexTrainingTrue)], y_train_val[np.logical_not(indexTrainingTrue)]),   # The same selection, but on the labels
    batch_size = batch_size,
    epochs = epochs,
    callbacks = callbacks,
  ).history

  # Evaluate the model on the validation data for this fold
  score = model.evaluate(x_train_val[np.logical_not(indexTrainingTrue)], y_train_val[np.logical_not(indexTrainingTrue)], verbose=0)
  # Append the score
  scores.append(score[1])

  # Calculate the best epoch for early stopping
  best_epoch = len(history['loss']) - patience
  best_epochs.append(best_epoch)

In [ ]:
# Print mean and standard deviation of MSE scores
print(f"Scores: {scores}. Mean: {np.mean(scores).round(4)}. Std:  {np.std(scores).round(4)}")

# Calculate the average best epoch from K folds cross-validation
avg_epochs = int(np.mean(best_epochs))
print(f"Best epochs: {best_epochs}. Best average epoch: {avg_epochs}")

In [ ]:
# Final training of the model

lr_scheduler = tfk.callbacks.ReduceLROnPlateau(
  monitor='val_accuracy',                                                 # Metric to monitor (validation mean squared error in this case)
  patience=5,                                                             # Number of epochs with no improvement after which learning rate will be reduced
  factor=0.9,                                                             # Factor by which the learning rate will be reduced (0.9 in this case)
  mode='min',                                                             # Mode to decide when to reduce learning rate ('min' means reduce when metric stops decreasing)
  min_lr=1e-5                                                             # Minimum learning rate
)

batch_size = 64

patience = 30

# We split the training_and_validation set in training set and validation set
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, random_state=seed, test_size=500, stratify = y_train_val)

callbacks = [lr_scheduler]

# Build a new model
model = build_model(1,64)

# Train the model on the whole training set
history = model.fit(
  x = x_train,
  y = y_train,
  validation_data=(x_val, y_val),
  batch_size = batch_size,
  epochs = avg_epochs,
  callbacks = callbacks,
).history

del avg_epochs, batch_size, callbacks, x_train, y_train

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(history['loss'], label='Training Loss', alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(history['val_loss'], label='Validation Loss', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Binary Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(history['accuracy'], label='Training Accuracy', alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(history['val_accuracy'], label='Validation Accuracy', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

del history

In [ ]:

# Validation Accuracy, F1, Precision, Recall computation

val_predictions = model.predict(x_val, verbose=0)
val_predictions = tf.where(val_predictions < 0.5, 0, 1)
val_predictions = val_predictions.numpy()

# Compute scores
val_accuracy = accuracy_score(y_val, val_predictions)
val_f1 = f1_score(y_val, val_predictions)
val_precision = precision_score(y_val, val_predictions)
val_recall = recall_score(y_val, val_predictions)

# Display the computed metrics
print('Val Accuracy:', val_accuracy.round(4))
print('Val F1:', val_f1.round(4))
print('Val Precision:', val_precision.round(4))
print('Val Recall:', val_recall.round(4))

# Test Accuracy, F1, Precision, Recall computation

test_predictions = model.predict(x_test, verbose=0)
test_predictions = tf.where(test_predictions < 0.5, 0, 1)
test_predictions = test_predictions.numpy()

# Compute scores
test_accuracy = accuracy_score(y_test, test_predictions)
test_f1 = f1_score(y_test, test_predictions)
test_precision = precision_score(y_test, test_predictions)
test_recall = recall_score(y_test, test_predictions)

# Display the computed metrics
print('Test Accuracy:', test_accuracy.round(4))
print('Test F1:', test_f1.round(4))
print('Test Precision:', test_precision.round(4))
print('Test Recall:', test_recall.round(4))

# Save the model
model.save("ResNetFrom0")